# Agent with tools
This notebook illustrates how to build a simple agent capable of using tools. 
We'll use crewAI to illustrate how agentics implements logical transduction

## Initialize environment

Note: Before executing this notebook on colab, copy your .env file in /content/drive/MyDrive/.env or otherwise you'll be prompted to enter your GEMINI_API_KEY all the times.

In [1]:
! uv pip install agentics-py

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    load_dotenv("/content/drive/MyDrive/.env")
    from google.colab import drive

    drive.mount("/content/drive")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Using Python 3.12.11 environment at: /Users/gliozzo/miniforge3/envs/agentics
Audited 1 package in 17ms
In Colab: False


## Create a new tool
Wrap the Duck Duck Go Search API call into a CrewAI tool using the @tool decorator. Similarly you can turn any function of your choice into a tool. 

In [2]:
from agentics.core.llm_connections import get_llm_provider
from crewai.tools import tool
from ddgs import DDGS

## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("fetch_data_async")
def web_search(query: str) -> str:
    """ Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))    

2025-09-05 16:14:38.683 | DEBUG    | agentics.core.llm_connections:<module>:90 - AGENTICS is connecting to the following LLM API providers:
2025-09-05 16:14:38.684 | DEBUG    | agentics.core.llm_connections:<module>:93 - 0 - WatsonX
2025-09-05 16:14:38.684 | DEBUG    | agentics.core.llm_connections:<module>:98 - 1 - Gemini
2025-09-05 16:14:38.684 | DEBUG    | agentics.core.llm_connections:<module>:102 - 2 - OpenAI
2025-09-05 16:14:38.684 | DEBUG    | agentics.core.llm_connections:<module>:104 - Please add API keys in .env file to add or disconnect providers.
2025-09-05 16:14:38.690 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-05 16:14:38.690 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'


## Define an Agent

Use crew AI agent directly create a document searcher agent, who is instructed to use the web_search tool defined above. Note that reasoning steps and other parameters can be customized. 


In [3]:
from crewai import Agent
doc_agent = Agent(
    role="Doc Searcher",
    goal="Find answers to questions from the user using the available web search tool.",
    backstory="A helpful assistant for extensive web search reports.",
    tools=[web_search],
    reasoning=True,
    reasoning_steps=2,
    verbose=True,
    llm=get_llm_provider()
)

2025-09-05 16:14:38.711 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-05 16:14:38.711 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'


## Define a Task

In [4]:
from crewai import Task

doc_task = Task(
    description="""Your task is to perform an extensive web search about
    the following question {question} and return a document providing answers to 
    the questions that explore several interesting aspects, each of them supported 
    by pertinent information from web search.  """,
    expected_output="""A markdown document with introduction and different sections""",
    agent=doc_agent,
    markdown=True
)

## Define a Crew

Crews are enviroments in which tasks are assigned to different agents following different policies. In this simple example we define a crew with a single agent and single task. 

In [5]:
from crewai import Crew

crew = Crew(
    agents=[doc_agent],
    tasks=[doc_task],
    verbose=True,
)

## Kickoff the crew

Crews are ultimatelly analoguous to very sophisticated function definitions, whose inputs are json dictonaries. In order to put them in action you should kickoff them providing input parameters.

In the Agent Trace stack you can see the plan made by the agent and the different execution steps which might involve tool calls.

In [6]:
result = crew.kickoff(inputs={"question": input("Ask your question> ") })

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 21b1a38d-11e5-4e8f-a91f-f70ab201bf8e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/gliozzo/miniforge3/envs/agentics/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/gliozzo/miniforge3/envs/agentics/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭─────────────────────────────────────────────── 🧠 Reasoning Plan ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ## Task Understanding and Plan                                                                                 │
│                                                                                                                 │
│  ### 1. Understanding of the Task                                                                               │
│  As a Doc Searcher, my task is to perform an extensive web search about Alfio Gliozzo and create a markdown     │
│  document that provides answers to questions exploring several interesting aspects of this individual. The      │
│  document should be well-structured, including an introduction and different sections, each supported by        │
│  pertinent information gathered from the web search.                                                            │
│                                                                                                                 │
│  ### 2. Key Steps to Complete the Task                                                                          │
│  To complete this task, I will follow these key steps:                                                          │
│  - **Step 1: Initial Research** - Use the fetch_data_async tool to gather initial information about Alfio       │
│  Gliozzo. This step will help in understanding who he is, his profession, achievements, and any other basic     │
│  information available.                                                                                         │
│  - **Step 2: In-Depth Search** - Perform a more in-depth search to explore various aspects of Alfio Gliozzo's   │
│  life and career. This could include his early life, education, career milestones, notable works or             │
│  achievements, and any challenges he might have faced.                                                          │
│  - **Step 3: Organization of Information** - Organize the gathered information into a structured format. This   │
│  will involve categorizing the information into different sections or topics, ensuring that the document flows  │
│  logically and is easy to follow.                                                                               │
│  - **Step 4: Creation of the Markdown Document** - Create the markdown document, incorporating the              │
│  introduction, various sections, and conclusions based on the information gathered. Ensure that the document    │
│  is well-formatted and readable.                                                                                │
│  - **Step 5: Review and Revision** - Review the document for accuracy, completeness, and clarity. Revise as     │
│  necessary to ensure that the document meets the expected standards and provides comprehensive answers to the   │
│  questions about Alfio Gliozzo.                                                                                 │
│                                                                                                                 │
│  ### 3. Approach to Challenges                                                                                  │
│  Challenges might arise in the form of limited information availability, difficulty in verifying the accuracy   │
│  of information, or organizing the information in a coherent manner. To address these challenges:               │
│  - **Information Availability**: Utilize multiple sources and databases to gather as much information as        │
│  possible. Sometimes, less popular or niche sources may provide valuable insights.                              │
│  - **Verification of Information**: Cross-verify information across different sources to ensure accuracy. This  │
│  step is crucial in maintaining the credibility of the 

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Doc Searcher                                                                                            │
│                                                                                                                 │
│  Task: Your task is to perform an extensive web search about                                                    │
│      the following question Who is Alfio gliozzo and return a document providing answers to                     │
│      the questions that explore several interesting aspects, each of them supported                             │
│      by pertinent information from web search.                                                                  │
│                                                                                                                 │
│  Reasoning Plan:                                                                                                │
│  ## Task Understanding and Plan                                                                                 │
│                                                                                                                 │
│  ### 1. Understanding of the Task                                                                               │
│  As a Doc Searcher, my task is to perform an extensive web search about Alfio Gliozzo and create a markdown     │
│  document that provides answers to questions exploring several interesting aspects of this individual. The      │
│  document should be well-structured, including an introduction and different sections, each supported by        │
│  pertinent information gathered from the web search.                                                            │
│                                                                                                                 │
│  ### 2. Key Steps to Complete the Task                                                                          │
│  To complete this task, I will follow these key steps:                                                          │
│  - **Step 1: Initial Research** - Use the fetch_data_async tool to gather initial information about Alfio       │
│  Gliozzo. This step will help in understanding who he is, his profession, achievements, and any other basic     │
│  information available.                                                                                         │
│  - **Step 2: In-Depth Search** - Perform a more in-depth search to explore various aspects of Alfio Gliozzo's   │
│  life and career. This could include his early life, education, career milestones, notable works or             │
│  achievements, and any challenges he might have faced.                                                          │
│  - **Step 3: Organization of Information** - Organize the gathered information into a structured format. This   │
│  will involve categorizing the information into different sections or topics, ensuring that the document flows  │
│  logically and is easy to follow.                                                                               │
│  - **Step 4: Creation of the Markdown Document** - Create the markdown document, incorporating the              │
│  introduction, various sections, and conclusions based on the information gathered. Ensure that the document    │
│  is well-formatted and readable.                                                                                │
│  - **Step 5: Review and Revision** - Review the document for accuracy, completeness, and clarity. Revise as     │
│  necessary to ensure that the document meets the expected standards and provides comprehensive answers to the   │
│  questions about Alfio Gliozzo.                        

...

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"Alfio Gliozzo biography\"}"                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  [{'title': 'ACM Digital Library Alfio Massimiliano Gliozzo - Home', 'href':                                    │
│  'https://dl.acm.org/profile/81100464565', 'body': 'Alfio Gliozzo · IBM Research AI, Yorktown Heights, NY, USA  │
│  · November 2023The Semantic Web – ISWC 2023https://doi.org/10.1007/978-3-031-47240-4_27 · research-article ·   │
│  Gaetano Rossiello · IBM Research AI, Thomas J. Watson Research Center, NY, Md. Faisal Mahbub Chowdhury ·'},    │
│  {'title': 'Alfio Massimiliano Gliozzo - Chief Science Catalyst', 'href':                                       │
│  'https://www.linkedin.com/in/gliozzo', 'body': 'Alfio Gliozzo is a Principal Researcher at IBM Research AI ,   │
│  where he serves as Technical Program Manager and Chief Science Catalyst for Data and AI Platforms.'},          │
│  {'title': 'Alfio Gliozzo - LinkedIn, Twitter - Clay.earth', 'href':                                            │
│  'https://clay.earth/profile/alfio-gliozzo', 'body': 'Alfio Massimiliano Gliozzo is a Principal Researcher at   │
│  IBM Research AI , serving as Technical Program Manager and Chief Science Catalyst for Data and AI.'},          │
│  {'title': 'Alfio Gliozzo | Scholar Profiles and Rankings', 'href':                                             │
│  'https://scholargps.com/scholars/81406211210414/alfio-gliozzo', 'body': 'Alfio Gliozzo . ScholarGPS® ID:       │
│  81406211210414. Affiliation. International Business Machines Corporation, Armonk, New York, United States.     │
│  Affiliation History.'}, {'title': 'Alfio Gliozzo', 'href': 'https://aclanthology.org/people/alfio-gliozzo/',   │
│  'body': 'Alfio Gliozzo ; 2025 · Knowledge Base Construction for Knowledge-Augmented Text-to-SQL ; 2023 ·       │
│  Retrieval-Based Transformer for Table Augmentation ; 2022 · KGI: An ...'}, {'title': 'LinkedIn Alfio           │
│  Massimiliano Gliozzo - Chief Science Catalyst - IBM | LinkedIn', 'href':                                       │
│  'https://www.linkedin.com/in/gliozzo/', 'body': 'July 19, 2024 - Chief Science Catalyst - IBM research ·       │
│  Alfio Gliozzo is a Principal Researcher at IBM Research AI, where he serves as Technical Program Manager and   │
│  Chief Science Catalyst for Data and AI Platforms. He leads a major research initiative focused on applying     │
│  Generative AI to transform business ...'}, ...                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Doc Searcher                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The above markdown document provides a comprehensive overview of Alfio Gliozzo's biography, career, research   │
│  interests, publications, and social media presence.                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b96eff68-6321-41b9-ba0e-ee843f25b5ba                                                                     │
│  Agent: Doc Searcher                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 21b1a38d-11e5-4e8f-a91f-f70ab201bf8e                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The above markdown document provides a comprehensive overview of Alfio Gliozzo's biography,      │
│  career, research interests, publications, and social media presence.                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Display the results as markdown

In [7]:
from IPython.display import Markdown, display

display(Markdown(result.raw))

The above markdown document provides a comprehensive overview of Alfio Gliozzo's biography, career, research interests, publications, and social media presence.